I cannot train model properly.......

### Reference
> @ragnar123 [bert-baseline](https://www.kaggle.com/ragnar123/bert-baseline)

Thank you for sharing

In [ ]:
import os
import math
import random
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from transformers import AutoTokenizer, TFAutoModel 
warnings.simplefilter('ignore')

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
SEED = 42
EPOCHS = 20
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
TFM_PATH = 'bert-base-uncased'
TOKENIZER_PATH = 'bert-base-uncased'
LR = 1e-3

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Constant
from tensorflow.python.keras.utils import tf_utils


def _resolve_training(layer, training):
    if training is None:
        training = K.learning_phase()
    if isinstance(training, int):
        training = bool(training)
    if not layer.trainable:
        # When the layer is not trainable, override the value
        training = False
    return training#tf_utils.constant_value(training)

In [ ]:
import tensorflow.keras.layers
class AdaCos(Layer):
    """
    Implementation of AdaCos layer. Reference: https://arxiv.org/abs/1905.00292
    
    Arguments:
      num_classes: number of classes to classify
      is_dynamic: if False, use Fixed AdaCos. Else, use Dynamic Adacos.
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 is_dynamic=True,
                 regularizer=None,
                 **kwargs):

        super().__init__(**kwargs)
        self._n_classes = num_classes
        self._init_s = math.sqrt(2) * math.log(num_classes - 1)
        self._is_dynamic = is_dynamic
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(name='W',shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer)
        if self._is_dynamic:
            self._s = self.add_weight(name='S',shape=(),
                                      initializer=Constant(self._init_s),
                                      trainable=False,
                                      aggregation=tf.VariableAggregation.MEAN)

    def call(self, inputs, training=None):
        embedding, label = inputs
        label = tf.cast(label, dtype=tf.int32)

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1])

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1)
        w = tf.nn.l2_normalize(self._w, axis=0)
        logits = tf.matmul(x, w)

        # Fixed AdaCos
        is_dynamic = self._is_dynamic#tf_utils.constant_value(self._is_dynamic)
        if not is_dynamic:
            # _s is not created since we are not in dynamic mode
            output = tf.multiply(self._init_s, logits)
            return output

        training = _resolve_training(self, training)
        if not training:
            # We don't have labels to update _s if we're not in training mode
            return self._s * logits
        else:
            theta = tf.math.acos(
                    K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            one_hot = tf.one_hot(label, depth=self._n_classes)
            b_avg = tf.where(one_hot < 1.0,
                             tf.exp(self._s * logits),
                             tf.zeros_like(logits))
            b_avg = tf.reduce_mean(tf.reduce_sum(b_avg, axis=1))
            theta_class = tf.gather_nd(
                    theta,
                    tf.stack([
                        tf.range(tf.shape(label)[0]),
                        tf.cast(label, tf.int32)
                    ], axis=1))
            mid_index = tf.shape(theta_class)[0] // 2 + 1
            theta_med = tf.nn.top_k(theta_class, mid_index).values[-1]

            # Since _s is not trainable, this assignment is safe. Also,
            # tf.function ensures that this will run in the right order.
            self._s.assign(
                    tf.math.log(b_avg) /
                    tf.math.cos(tf.minimum(math.pi/4, theta_med)))

            # Return scaled logits
            return self._s * logits

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
seed_everything(SEED)

In [ ]:
train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
N_CLASSES = train_df['label_group'].nunique()

In [ ]:
#b=TFAutoModel.from_pretrained(TFM_PATH)
#b.save_pretrained('base_bert')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
tokenizer.save_pretrained('tokenizer')

In [ ]:
def load_df():
    train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
    train_df['label_group'] = LabelEncoder().fit_transform(train_df['label_group'])
    N_CLASSES = train_df['label_group'].nunique()
    train_x, valid_x = train_test_split(train_df[['title', 'label_group']], shuffle=True, stratify=train_df['label_group'], random_state=SEED, test_size=0.33)
    return train_x, valid_x

In [ ]:
def tokenize(df):
    inputs = tokenizer(df.title.tolist(), return_tensors='tf', max_length=64, padding='max_length', truncation=True)
    return inputs['input_ids'].numpy(), inputs['attention_mask'].numpy()

In [ ]:
def load_ds(tokens, masks, labels, mode='train'):
    text_ds = tf.data.Dataset.from_tensor_slices((tokens, masks, labels))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((text_ds, label_ds))
    if mode == 'train':
        ds = ds.repeat()
        ds = ds.shuffle(len(tokens))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
def load():
    train_df, valid_df = load_df()
    STEPS_PER_EPOCH = train_df.shape[0] // BATCH_SIZE
    if train_df.shape[0] % BATCH_SIZE != 0: STEPS_PER_EPOCH += 1
    train_x, valid_x = tokenize(train_df), tokenize(valid_df)
    train_ds, valid_ds = load_ds(*train_x, train_df.label_group.values), load_ds(*valid_x, valid_df.label_group.values, mode='valid')
    return train_ds, valid_ds, STEPS_PER_EPOCH

In [ ]:
!git clone https://github.com/Jannoshh/simple-sam
!mv simple-sam sam
import sam
import sam.sam
from sam.sam import sam_train_step

In [ ]:
class RobertaArcFace(tf.keras.Model):
    def __init__(self,rho=0.05):
        super().__init__()
        self.roberta = TFAutoModel.from_pretrained(TFM_PATH)
        adacos=AdaCos(N_CLASSES, name='AdaCos' )    
        self.arc_margin =adacos
        self.softmax = tf.keras.layers.Softmax(dtype='float32')
        self.rho = tf.constant(rho, dtype='float32')
    def train_step(self, data):
        return sam_train_step(self, data)
    def call(self, inputs):
        tokens, masks, labels = inputs
        out = self.roberta(tokens, masks)
        feats = out.last_hidden_state[:, 0, :]
        out = self.arc_margin((feats, labels),True)
        out = self.softmax(out)
        return out

In [ ]:

class SAMModel(tf.keras.models.Model):
    def train_step(self, data):
        return sam_train_step(self, data)
def build_model():
    #model = RobertaArcFace()
    max_len = 64
    roberta=TFAutoModel.from_pretrained(TFM_PATH)
    adacos=AdaCos(N_CLASSES, name='AdaCos' ) 
    softmax= tf.keras.layers.Softmax(dtype='float32')
    tokens = tf.keras.layers.Input(shape = (max_len,), dtype=tf.int32, name = 'tokens')
    masks  = tf.keras.layers.Input(shape = (max_len,), dtype=tf.int32, name = 'masks')
    labels = tf.keras.layers.Input(shape = (), name = 'label')
    out = roberta(tokens, masks)
    feats = out.last_hidden_state[:, 0, :]
    out = adacos((feats, labels),False)
    out = softmax(out)
    model = SAMModel(inputs = [tokens,masks, labels], outputs = [out])
    radam = tfa.optimizers.RectifiedAdam()
    base_optimizer = tf.keras.optimizers.SGD()  # define an optimizer for the "sharpness-aware" update
    #sam_optimizer = sam.sam.SAM(base_optimizer)
    model.compile(optimizer=radam,
                  loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# Function for a custom learning rate scheduler with warmup and decay
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback



In [ ]:
def main():
    train_ds, valid_ds, STEPS_PER_EPOCH = load()
    checkpoint = ModelCheckpoint(
        f'bert-arcface.h5', 
        monitor = 'val_loss', 
        save_best_only = True,
        save_weights_only = True, 
        mode = 'min'
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00001,verbose=1)
   
    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                              patience = 3, mode = 'min', verbose = 1,
                           restore_best_weights = True)
    
    with strategy.scope():
        model = build_model()
    model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=[checkpoint,early_stop,reduce_lr ]
    )
    #model.save("hoge", save_format="tf")


In [ ]:
main()

In [ ]:
!zip base_bert.zip ./base_bert -r

In [ ]:
import os 
os.chdir('/kaggle/working')
from IPython.display import FileLink 
FileLink('./base_bert.zip')

In [ ]:
m=tf.keras.models.load_model('./bert-arcface.h5')

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
import math, re, os
import gc
import tensorflow as tf
#from tensorflow.keras.layers import *
import numpy as np
from tensorflow import keras
from functools import partial

from tensorflow.keras.applications.imagenet_utils import decode_predictions
import tensorflow_addons as tfa
FOLD=4
kf = KFold(n_splits=FOLD, shuffle=True, random_state = SEED)
#GCS_PATH = '/kaggle/input/cassava-leaf-disease-tfrecords-center-512x512'
i=0
#for n_fold, (train, test) in enumerate(cv.split(X)):
target=train_df[['title', 'label_group']]
def load_df():
    train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
    train_df['label_group'] = LabelEncoder().fit_transform(train_df['label_group'])
    N_CLASSES = train_df['label_group'].nunique()
    train_x, valid_x = train_test_split(train_df[['title', 'label_group']], shuffle=True, stratify=train_df['label_group'], random_state=SEED, test_size=0.33)
    return train_x, valid_x
for train_index, test_index in kf.split(target):
    K.clear_session()
    gc.collect()
    train=target.iloc[train_index]
    valid=target.iloc[test_index]
    STEPS_PER_EPOCH = train.shape[0]  // BATCH_SIZE
    if train_df.shape[0] % BATCH_SIZE != 0: STEPS_PER_EPOCH += 1
    train_x, valid_x = tokenize(train), tokenize(valid)
    train_ds, valid_ds = load_ds(*train_x, train.label_group.values), load_ds(*valid_x, valid.label_group.values, mode='valid')
    del train_x,valid_x,train,valid
    print(f'fold={i}')
    checkpoint = ModelCheckpoint(
        f'bert-adacos_fold{i}.h5', 
        monitor = 'val_loss', 
        save_best_only = True,
        save_weights_only = True, 
        mode = 'min'
    )
    #reduce_lr = ReduceLROnPlateau()
    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                              patience = 3, mode = 'min', verbose = 1,
                           restore_best_weights = True)
    
    with strategy.scope():
        model = build_model()
    model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=[checkpoint,early_stop,get_lr_callback() ]
    )
    del model
